# Adapter usage demonstration #

First create a couple of handy descriptors to allow us to constrain some properties.

Basically, compensate for the fact that Python isn't typed!

In [1]:
from numbers import Number


class NumberProperty(object):
    
    def __init__(self, base_attribute):
        self.base_attribute = base_attribute
        
        self.__doc__ = (
            "A property that can only be set to a number. "
            "The underlying storage attribute is '%s'" % base_attribute)
    
    def __get__(self, instance, owner):
        if instance is None: 
            return self
        
        return getattr(instance, self.base_attribute)
    
    def __set__(self, instance, value):
        if not isinstance(value, Number):
            raise TypeError('%r is not a Number' % value)
        
        return setattr(instance, self.base_attribute, value)
    
    
class RangeProperty(NumberProperty):
    
    def __init__(self, base_attribute, start, length):
        super(RangeProperty, self).__init__(base_attribute)
        self.start = start
        self.length = length
        
        self.__doc__ = (
            "A property that can only be set to a number between %f and %f. "
            "The underlying storage attribute is '%s'" % (
                start, start + length, base_attribute))
    
    def __set__(self, instance, value):
        if not isinstance(value, Number):
            raise TypeError('%r is not a Number' % value)
            
        if not (self.start <= value < (self.start + self.length)):
            raise ValueError(
                '%f is not in range %f -> %f' % (value, self.start, self.start + self.length))
        
        return setattr(instance, self.base_attribute, value)

Next, create two classes that have a relationship to each other. Note that:

- `ImperialWindMeasurement` has an extra property, which is a derived property calculated entirely from an actual data attribute.

In [ ]:
class MetricWindMeasurement(object):
    
    windspeed_m_per_s = NumberProperty('_windspeed')
    winddirection = RangeProperty('_winddirection', 0, 100)
    

class ImperialWindMeasurement(object):
    
    windspeed_knots = NumberProperty('_windspeed')
    winddirection = RangeProperty('_winddirection', 0, 360)
    
    @property
    def text_winddirection(self):
        wd = self.winddirection
        if (360 - 22.5) <= wd or wd < 22.5:
            return 'north'
    
        if 22.5 <= wd < 45 + 22.5:
            return 'north-east'
        
        if 45 + 22.5 <= wd < 90 + 22.5:
            return 'east'
        
        return "I'm bored"

Just illustrate that the type checking is working.

In [ ]:
mwm = MetricWindMeasurement()
mwm.windspeed_m_per_s = 'not a number'

Next, configure an adapter class. It's going to convert from metric to imperial.

In [ ]:
from sipparty.adapter import (
    ProxyAdapter, AdaptToClass, AdapterProperty, AdapterOptionKeyConversion)


class MetricToImperialWindMeasurementAdapter(ProxyAdapter):
    from_class = MetricWindMeasurement
    to_class = ImperialWindMeasurement
    adaptations = (
        ('windspeed_knots', 'windspeed_m_per_s', {
                AdapterOptionKeyConversion: lambda x: x * 1.943844
            }
        ),
        ('winddirection', 'winddirection', {
                AdapterOptionKeyConversion: lambda x: x * 3.6
            })
    )


Use it!

In [ ]:
mwm = MetricWindMeasurement()
mwm.windspeed_m_per_s = 3

imp_wm = AdaptToClass(mwm, ImperialWindMeasurement)
print('Windspeed in knots: %f' % imp_wm.windspeed_knots)

mwm.windspeed_m_per_s = 1.1
print('Windspeed in knots: %f' % imp_wm.windspeed_knots)

print('Is it of ImperialWindMeasurement class: %s' % isinstance(imp_wm, ImperialWindMeasurement))

mwm.winddirection = 13
print(imp_wm.text_winddirection)

To make it even neater, there's a descriptor to provide an interface directly to the alternative class.

In [ ]:
class MetricWithAdapter(MetricWindMeasurement):
    imperial_adapter = AdapterProperty(ImperialWindMeasurement)
    
    
mwa = MetricWithAdapter()
mwa.winddirection = 27
mwa.imperial_adapter.text_winddirection